In [ ]:
# Resume training

!git clone https://github.com/timothycao/cifar10_classifier.git
%cd cifar10_classifier

import torch
import torch.optim as optim
import torchvision.transforms as transforms
from train import main as train
from inference import main as inference

model_path = '/kaggle/input/resnet50/resnet50_3.pth'

def load_model(model_path, device):
    print(f'Loading model...')
    model = torch.load(model_path, map_location=device, weights_only=False)
    model.to(device)
    if not hasattr(model, 'name'):
        model.name = 'ResNetCustom'
    return model

# Training parameters
EPOCHS = 50
TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE = 100
AUGMENTATIONS = [
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomCrop(32, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    transforms.RandAugment(num_ops=2, magnitude=12),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
    transforms.RandomGrayscale(p=0.2)
]
SAVE_MODE = 'best'  # Options: 'best', 'every'
SAVE_EVERY_N = 1

# Load model
try:
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = load_model(model_path, device)
except AssertionError as e:
    print(f'Failed to load model: {e}')
    exit(1)

# Train model
try:
    OPTIMIZER = optim.SGD(model.parameters(), momentum=0.9, weight_decay=8e-4)
    SCHEDULER = optim.lr_scheduler.OneCycleLR(OPTIMIZER, max_lr=0.0035, total_steps=EPOCHS, final_div_factor=50)

    train(model, EPOCHS, train_batch_size=TRAIN_BATCH_SIZE, test_batch_size=TEST_BATCH_SIZE, augmentations=AUGMENTATIONS,
          optimizer=OPTIMIZER, scheduler=SCHEDULER, save=SAVE_MODE, every_n=SAVE_EVERY_N)
except (ValueError, TypeError) as e:
    print(f'Training failed: {e}')
    exit(1)

Cloning into 'cifar10_classifier'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 93 (delta 53), reused 68 (delta 28), pack-reused 0 (from 0)
Receiving objects: 100% (93/93), 24.05 KiB | 4.01 MiB/s, done.
Resolving deltas: 100% (53/53), done.
/kaggle/working/cifar10_classifier
Loading model...
Total model parameters: 4848027
Loading data...
Preprocessing pipeline:
 Compose(
    RandomHorizontalFlip(p=0.5)
    RandomCrop(size=(32, 32), padding=4)
    ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2), saturation=(0.8, 1.2), hue=(-0.1, 0.1))
    RandomAffine(degrees=[-10.0, 10.0], translate=(0.1, 0.1), scale=(0.8, 1.2))
    RandAugment(num_ops=2, magnitude=12, num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
    GaussianBlur(kernel_size=(3, 3), sigma=(0.1, 2.0))
    RandomGrayscale(p=0.2)
    ToTensor()
    Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.20

In [ ]:
# Run inference
try:
    inference('ResNetCustom_epoch50_acc95.pth')
except FileNotFoundError as e:
    print(f'Inference failed: {e}')

Loading data...
Loading model...
Running inference...
Predictions saved as ResNetCustom_epoch50_acc95.csv
